In [1]:
%tensorflow_version 1.x 
import keras
from keras.applications import ResNet50
from keras.models import Sequential
from keras.layers import Dense, Flatten, GlobalAveragePooling2D


TensorFlow 1.x selected.


Using TensorFlow backend.


In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import matplotlib.pyplot as plt
import os
from random import shuffle
from tqdm import tqdm
import tensorflow as tf
import cv2
import numpy as np 
from tensorflow.python.framework import graph_util
from keras.models import Sequential
from keras.layers import *
from keras.optimizers import *
from sklearn.model_selection import train_test_split

In [0]:
data1='/content/drive/My Drive/fintest/1coal'
data2='/content/drive/My Drive/fintest/2coal'
data3='/content/drive/My Drive/fintest/3coal'

In [0]:
def data_with_label():
    images=[]
    cnt = 0
    for i in tqdm(os.listdir(data1)):
        path=os.path.join(data1,i)
        img=cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        #img = tf.image.grayscale_to_rgb( img, name=None)
        img = cv2.cvtColor(img,cv2.COLOR_GRAY2RGB)
        img=cv2.resize(img,(224,224))
        images.append([np.array(img),np.array([1,0,0])])
    cnt = 0
    for i in tqdm(os.listdir(data2)):
        path=os.path.join(data2,i)
        img=cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        #img = tf.image.grayscale_to_rgb( img, name=None)
        img = cv2.cvtColor(img,cv2.COLOR_GRAY2RGB)
        img=cv2.resize(img,(224,224))
        images.append([np.array(img),np.array([0,1,0])])
    cnt = 0
    for i in tqdm(os.listdir(data3)):
        path=os.path.join(data3,i)
        img=cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        #img = tf.image.grayscale_to_rgb( img, name=None)
        img = cv2.cvtColor(img,cv2.COLOR_GRAY2RGB)
        img=cv2.resize(img,(224,224))
        images.append([np.array(img),np.array([0,0,1])])
        
    shuffle(images)
    return images

In [6]:
data = data_with_label()

100%|██████████| 465/465 [05:18<00:00,  1.46it/s]


In [0]:
import random
random.shuffle(data)
train_data, test_data = train_test_split(data, test_size=0.2)

In [0]:
from keras.applications.resnet50 import preprocess_input
tr_img_data = np.array([i[0] for i in train_data]).reshape(-1,224,224,3)
tr_img_data = preprocess_input(tr_img_data)
tr_lbl_data = np.array([i[1] for i in train_data])
tst_img_data = np.array([i[0] for i in test_data]).reshape(-1,224,224,3)
tst_img_data = preprocess_input(tst_img_data)
tst_lbl_data = np.array([i[1] for i in test_data])

In [9]:
from keras.applications.resnet50 import ResNet50
from keras.models import Model
import keras

resnet_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


Instructions for updating:
If using Keras pass *_constraint arguments to layers.



/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


94658560/94653016 [==============================] - 3s 0us/step


In [0]:
model = Sequential()
model.add(resnet_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(256, activation='relu'))
model.add(Dropout(.25))
model.add(BatchNormalization())
model.add(Dense(3, activation='softmax'))

In [0]:
for layer in resnet_model.layers:
    if isinstance(layer, BatchNormalization):
        layer.trainable = True
    else:
        layer.trainable = False

In [0]:
from keras import optimizers
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-5),
              metrics=['accuracy'])

In [13]:
import keras 
checkpoint_path = "/content/drive/My Drive/k40.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
#60
# Create a callback that saves the model's weights
cp_callback = keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# Train the model with the new callback
history = model.fit(tr_img_data, 
          tr_lbl_data,  
          epochs=50,
          validation_data=(tst_img_data,tst_lbl_data),
          callbacks=[cp_callback])  # Pass callback to training


Train on 923 samples, validate on 231 samples
Epoch 1/50
923/923 [==============================] - 18s 19ms/step - loss: 1.4806 - accuracy: 0.3586 - val_loss: 1.3263 - val_accuracy: 0.4286

Epoch 00001: saving model to /content/drive/My Drive/k40.ckpt
Epoch 2/50
923/923 [==============================] - 6s 6ms/step - loss: 1.2732 - accuracy: 0.4247 - val_loss: 1.2109 - val_accuracy: 0.4416

Epoch 00002: saving model to /content/drive/My Drive/k40.ckpt
Epoch 3/50
923/923 [==============================] - 6s 6ms/step - loss: 1.1936 - accuracy: 0.4518 - val_loss: 1.1356 - val_accuracy: 0.4848

Epoch 00003: saving model to /content/drive/My Drive/k40.ckpt
Epoch 4/50
923/923 [==============================] - 6s 6ms/step - loss: 1.0333 - accuracy: 0.5352 - val_loss: 1.0634 - val_accuracy: 0.5022

Epoch 00004: saving model to /content/drive/My Drive/k40.ckpt
Epoch 5/50
923/923 [==============================] - 6s 6ms/step - loss: 0.9717 - accuracy: 0.5536 - val_loss: 1.0003 - val_accura